In [5]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tools import diffusionmap, get_acc
import time
import os
import pandas as pd

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
from sklearn.neighbors import NearestNeighbors

In [17]:
current_dir = os.getcwd()

# Constants
Add_noise = [0, 1]
N = 60399
embed_dim = 12
kmean_iter = 12

In [13]:
# Load data
data = (pd.read_csv("Patient1Values.csv")).to_numpy()
data_new = np.array(data[:, 3:10], dtype=float) #feature vectors
label = np.array(data[:,10:11], dtype=int) #last col is the activity label
data_new = data_new[:N]
label = label[:N].flatten()
sorted_indices = label.argsort()
data_new = data_new[sorted_indices]
label = label[sorted_indices]


In [10]:
data_new.shape[0]

60399

In [ ]:
# Add noise
iterations = 0
for add_noise in Add_noise:
    print(iterations)
    if add_noise:
        clean_data = data_new.copy()
        noise = 0.2 * np.random.randn(*data_new.shape)  # Gaussian noise
        # noise = np.random.standard_t(4, size=data.shape) / np.sqrt(2) * 0.2  # Student-t noise
        data_new += noise

    # DM & accuracy
    start_time = time.time()
    
    #Diffusion Map Function
    print('Got here')
    #dm_embed, _ = diffusionmap.Diffusion_Map(data_new, embed_dim, 12, False)
    
#def Diffusion_Map(data, dim, knn, denoise=False):
    denoise = False
    n = data_new.shape[0]

    # Search for K nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=12, algorithm='auto').fit(data_new)
    distances, indices = nbrs.kneighbors(data_new)

    # Make sure d(x,x)=0
    distances[:, 0] = 0

    # Find sigma
    sig = np.percentile(distances[:, -1] ** 2, 50)
    
    # Affinity matrix
    ker = np.exp(-3.5 * (distances ** 2) / sig)

    ii = np.repeat(np.arange(n)[:, np.newaxis], embed_dim, axis=1)
    
    print('Got here')
    W = csr_matrix((ker.ravel(), (ii.ravel(), indices.ravel())), shape=(n, n))
    print('Got here')
    W = W.maximum(W.transpose())

    if denoise:
        W.setdiag(0)
        W.eliminate_zeros()

    # Graph Laplacian
    D = np.sqrt(W.sum(axis=1).A1)  # Convert sparse matrix sum to 1D numpy array
    D_inv = 1 / D #Could this be giving us issues
    W = W.multiply(D_inv[:, np.newaxis]).multiply(D_inv)
    print('Got here')
    # Ensure symmetry
    W = (W + W.T) / 2
    print('Got here')

    # Eigenvalue Decomposition
    print(W.shape)
    S, U = eigs(W, k=embed_dim + 1)
    print('Got here')
    U = U[:, 1:].real
    print('Got here')
    S = np.diag(S[1:].real)
    print('Got here')
    U = U / D[:, np.newaxis]
    
    dm_embed, _ = U,S
    
    
    #Diffusion map ends
    print('Got here')
    time_dm = time.time() - start_time

    # Calculate accuracy
    acc_dm = get_acc.get_acc(dm_embed, label, kmean_iter)

    # Save files
    dm_embed = dm_embed[:, :3]
    if add_noise:
        path = os.path.join(current_dir, 'noisy_data', 'dm')
        os.makedirs(path, exist_ok=True)
        os.chdir(path)
    else:
        path = os.path.join(current_dir, 'clean_data', 'dm')
        os.makedirs(path, exist_ok=True)
        os.chdir(path)

    np.save('label.npy', label)
    np.save('dm_embed.npy', dm_embed)
    np.save('acc_dm.npy', acc_dm)
    np.save('time_dm.npy', time_dm)
    iterations += 1

0
Got here
Got here
Got here
Got here
Got here
(60399, 60399)


In [15]:
# Add noise
iterations = 0
for add_noise in Add_noise:
    print(iterations)
    if add_noise:
        clean_data = data_new.copy()
        noise = 0.2 * np.random.randn(*data_new.shape)  # Gaussian noise
        # noise = np.random.standard_t(4, size=data.shape) / np.sqrt(2) * 0.2  # Student-t noise
        data_new += noise

    # DM & accuracy
    start_time = time.time()
    dm_embed, _ = diffusionmap.Diffusion_Map(data_new, embed_dim, 200, False)
    time_dm = time.time() - start_time

    # Calculate accuracy
    acc_dm = get_acc.get_acc(dm_embed, label, kmean_iter)

    # Save files
    dm_embed = dm_embed[:, :3]
    if add_noise:
        path = os.path.join(current_dir, 'noisy_data', 'dm')
        os.makedirs(path, exist_ok=True)
        os.chdir(path)
    else:
        path = os.path.join(current_dir, 'clean_data', 'dm')
        os.makedirs(path, exist_ok=True)
        os.chdir(path)

    np.save('label.npy', label)
    np.save('dm_embed.npy', dm_embed)
    np.save('acc_dm.npy', acc_dm)
    np.save('time_dm.npy', time_dm)
    iterations += 1

0


KeyboardInterrupt: 